<a href="https://colab.research.google.com/github/dasalgadob/pythonAlgoritmhs/blob/master/cryptography/Block_Cipher_AES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://en.wikipedia.org/wiki/Rijndael_S-box
SBOX = [['63', '7c', '77', '7b', 'f2', '6b', '6f', 'c5', '30', '01', '67', '2b', 'fe', 'd7', 'ab', '76'],
        ['ca', '82', 'c9', '7d', 'fa', '59', '47', 'f0', 'ad', 'd4', 'a2', 'af', '9c', 'a4', '72', 'c0'],
        ['b7', 'fd', '93', '26', '36', '3f', 'f7', 'cc', '34', 'a5', 'e5', 'f1', '71', 'd8', '31', '15'],
		['04', 'c7', '23', 'c3', '18', '96', '05', '9a', '07', '12', '80', 'e2', 'eb', '27', 'b2', '75'],
		['09', '83', '2c', '1a', '1b', '6e', '5a', 'a0', '52', '3b', 'd6', 'b3', '29', 'e3', '2f', '84'],
		['53', 'd1', '00', 'ed', '20', 'fc', 'b1', '5b', '6a', 'cb', 'be', '39', '4a', '4c', '58', 'cf'],
		['d0', 'ef', 'aa', 'fb', '43', '4d', '33', '85', '45', 'f9', '02', '7f', '50', '3c', '9f', 'a8'],
		['51', 'a3', '40', '8f', '92', '9d', '38', 'f5', 'bc', 'b6', 'da', '21', '10', 'ff', 'f3', 'd2'],
		['cd', '0c', '13', 'ec', '5f', '97', '44', '17', 'c4', 'a7', '7e', '3d', '64', '5d', '19', '73'],
		['60', '81', '4f', 'dc', '22', '2a', '90', '88', '46', 'ee', 'b8', '14', 'de', '5e', '0b', 'db'],
		['e0', '32', '3a', '0a', '49', '06', '24', '5c', 'c2', 'd3', 'ac', '62', '91', '95', 'e4', '79'],
		['e7', 'c8', '37', '6d', '8d', 'd5', '4e', 'a9', '6c', '56', 'f4', 'ea', '65', '7a', 'ae', '08'],
		['ba', '78', '25', '2e', '1c', 'a6', 'b4', 'c6', 'e8', 'dd', '74', '1f', '4b', 'bd', '8b', '8a'],
		['70', '3e', 'b5', '66', '48', '03', 'f6', '0e', '61', '35', '57', 'b9', '86', 'c1', '1d', '9e'],
		['e1', 'f8', '98', '11', '69', 'd9', '8e', '94', '9b', '1e', '87', 'e9', 'ce', '55', '28', 'df'],
		['8c', 'a1', '89', '0d', 'bf', 'e6', '42', '68', '41', '99', '2d', '0f', 'b0', '54', 'bb', '16']]

R_CON = ['01000000', '02000000', '04000000', '08000000', '10000000',
         '20000000', '40000000', '80000000', '1b000000', '36000000']

MC_MATRIX = ['02030101', '01020301', '01010203', '03010102']
MC_MATRIX_INV = ['0e0b0d09', '090e0b0d', '0d090e0b', '0b0d090e']

def gf_256(a, b):
    a=int(a,16)
    b=int(b,16)
    p = 0
    hiBitSet = 0
    for i in range(8):
        if b & 1 == 1:
            p ^= a
        hiBitSet = a & 0x80
        a <<= 1
        if hiBitSet == 0x80:
            a ^= 0x1b
        b >>= 1
    return p % 256

def check(name, x):
    print name, x, len(x)
    
def iterate(it, n):
    for i in range(0, len(it), n):
        yield it[i:i+n]

In [4]:
def to_matrix(string):
    byte = list(iterate(string, 2))
    return [''.join(byte[i::4]) for i in range(4)]
        
check('to_matrix', to_matrix('2b7e151628aed2a6abf7158809cf4f3c'))
    
def rot_word(word, i):
    i *= 2
    return word[i:] + word[:i]

check('rot_word', rot_word('09cf4f3c', 1))


def sub_word(word, table):
    return ''.join(table[int(a, 16)][int(b, 16)] for a, b in iterate(word, 2))
    
check('sub_word', sub_word('cf4f3c09', SBOX))
    
def xor(x, y):
    return format(int(x, 16) ^ int(y, 16), '08x')
    
check('xor', xor('8a84eb01', '01000000'))

def ark(x, y):
    return [xor(a, b) for a, b in zip(x, y)]
    
check('ark', ark(['41657561', '45737963', '53202069', '206d666c'], ['2b28ab09', '7eaef7cf', '15d2154f', '16a6883c']))

def sb(state, table):
    return [sub_word(a, table) for a in state]
    
check('sb', sb(['6a4dde68', '3bdd8eac', '46f23526', '36cbee50'], SBOX))
#19617b67467c866ec20e2e1fc9e08b8e
check('sb', sb(['19617b67', '467c866e', 'c20e2e1f', 'c9e08b8e'], SBOX))
#d4ef21855a10449f25ab31c0dde13d19

def sr(state):
    return [rot_word(word, i) for i, word in enumerate(state)]
    
check('sr', sr(['02e31d45', 'e2c11991', '5a8996f7', '051f2853']))
#19617b67467c866ec20e2e1fc9e08b8e
check('sr', sr(['19617b67', '467c866e', 'c20e2e1f', 'c9e08b8e']))
#sr ['19617b67', '7c866e46', '2e1fc20e', '8ec9e08b'] 4
#19617b677c866e462e1fc20e8ec9e08b



# x: (xr, xc) y: (yr, yc) xy: (xr, yc)
def gf_mul(x, y):
    result = [''] * len(x)
    for i in range(len(x)):
        for j in range(0, len(x[0]), 2):
            cell = 0
            for k in range(len(y)):
                cell ^= gf_256(x[i][2*k:2*k+2], y[k][j:j+2])
            result[i] += format(cell, '02x')
    return result
    
check('gf_mul', gf_mul(MC_MATRIX, ['02e31d45', 'c11991e2', '96f75a89', '53051f28']))
#19617b67467c866ec20e2e1fc9e08b8e
check('gf_mul', gf_mul(MC_MATRIX, ['19617b67', '467c866e', 'c20e2e1f', 'c9e08b8e']))
#gf_mul ['f3a8c2ed', '016b9514', '803a27be', '260a28df'] 4
#f3a8c2ed016b9514803a27be260a28df

to_matrix ['2b28ab09', '7eaef7cf', '15d2154f', '16a6883c'] 4
rot_word cf4f3c09 8
sub_word 8a84eb01 8
xor 8b84eb01 8
ark ['6a4dde68', '3bdd8eac', '46f23526', '36cbee50'] 4
sb ['02e31d45', 'e2c11991', '5a8996f7', '051f2853'] 4
sb ['d4ef2185', '5a10449f', '25ab31c0', 'dde13d19'] 4
sr ['02e31d45', 'c11991e2', '96f75a89', '53051f28'] 4
sr ['19617b67', '7c866e46', '2e1fc20e', '8ec9e08b'] 4
gf_mul ['9904d716', '69d6d532', '010019d6', 'f7dad2f4'] 4
gf_mul ['f3a8c2ed', '016b9514', '803a27be', '260a28df'] 4


In [7]:
## 8 point
key = '9a2bda0ee132ce621fbfacd1896f4721'
k = to_matrix(key)
print(k)
for i in range(10):
    for j in range(4):
        ki = k[-1]
        
        if j == 0:
            ki = rot_word(ki, 1)
            ki = sub_word(ki, SBOX)
            ki = xor(ki, R_CON[i])
            
        ki = xor(ki, k[-4])
        k.append(ki)
        
k = [''.join(k[i:i+4]) for i in range(0, len(k), 4)]
k = [to_matrix(i) for i in k]
k

['9ae11f89', '2b32bf6f', 'daceac47', '0e62d121']


[['9a2bda0e', 'e132ce62', '1fbfacd1', '896f4721'],
 ['311ac0ce', 'dfed2341', 'e25df120', '224d0a2b'],
 ['b0aa6aa4', '6885a6e7', '134ebf9f', 'a9e4eec5'],
 ['208ae044', 'b3369077', 'b5fb44db', 'e004ea2f'],
 ['dd57b7f3', '0a3cacdb', 'a05b1fc4', 'fbff153a'],
 ['74239467', '162a865d', '207b64a0', 'f6091c26'],
 ['183bafc8', 'f6dc5a07', 'd7acc868', '737a6640'],
 ['9da609c1', 'b36f3532', 'de72bad2', '9be187c7'],
 ['3e989150', '06695c6e', '186ad002', 'e3028542'],
 ['ba22b3e3', '7118442a', '345e8e8c', 'b0b23775'],
 ['694bf81b', '150d4963', 'a9f779f5', 'a1132451']]

In [6]:
33d2cd448bb9066927e94502a9cb1a3b

SyntaxError: ignored

In [1]:
key = '9a2bda0ee132ce621fbfacd1896f4721'
k = list(iterate(key, 8))

for i in range(10):
    for j in range(4):
        ki = k[-1]
        
        if j == 0:
            ki = rot_word(ki, 1)
            ki = sub_word(ki, SBOX)
            ki = xor(ki, R_CON[i])
            
        ki = xor(ki, k[-4])
        k.append(ki)
        
k = [''.join(k[i:i+4]) for i in range(0, len(k), 4)]
k = [to_matrix(i) for i in k]
k

NameError: ignored

In [8]:
#a088232afa54a36cfe2c397617b13905
print(k[0])
#key = 9a2bda0ee132ce621fbfacd1896f4721
print(m)
print(ark(m, ['9a2bda0e', 'e132ce62', '1fbfacd1', '896f4721']))
#rta =  9a2bda0ce132ce691fbfacd6896f472f

['2b28ab09', '7eaef7cf', '15d2154f', '16a6883c']


NameError: ignored

In [9]:
def encrypt(m, k):
    m = to_matrix(m)
    
    #Stage 1
    c = ark(m, k[0])
    print(c)
    
    # Stage 2-3
    for i in range(1, len(k)):
        c = sb(c, SBOX)
        c = sr(c)
        c = gf_mul(MC_MATRIX, c) if i != len(k) - 1 else c
        c = ark(c, k[i])
        print(c)
    
    return ''.join(to_matrix(''.join(c)))

m = '2b7e151628aed2a6abf7158809cf4f3c'
print(encrypt(m, k))

['00000000', '00000000', '00000000', '00000000']
['c3eb4049', '9937c00f', '9d4f5a15', '74d25a66']
['96ad3a16', '2969878c', '13f587e9', 'c2bf383f']
['7474dcf6', 'b5aae45a', '9f2f78b4', 'd2d9d6bf']
['8b1496d8', '422bd331', 'e065ef11', 'ec47a5f1']
['d875e2ec', 'cee66f11', '5757cdc8', 'c8f16a72']
['dbd39792', '7216322c', 'ede3d266', '6aca7260']
['8bc73d41', '7728471a', '13ec8031', '974dfd4f']
['85961cfc', '4fca37e2', '6ccab424', '8cef0b22']
['1bc537e4', '177b4948', '2a16f7a1', '95ee9303']
['7f6f7bdf', '35d56d93', '9117e94b', 'd3a3e07a']
7f3591d36fd517a37b6de9e0df934b7a


In [10]:
def inv(table):
    table_inv = [['' for j in range(len(table[0]))] for i in range(len(table))]
    
    for i in range(len(table)):
        for j in range(len(table[0])):
            new_i, new_j = table[i][j]
            new_i, new_j = int(new_i, 16), int(new_j, 16)
            table_inv[new_i][new_j] = hex(i)[2:] + hex(j)[2:]  
    
    return table_inv

SBOX_INV = inv(SBOX)
print SBOX_INV

def sr_inv(state):
    return [rot_word(word, -i) for i, word in enumerate(state)]

[['52', '09', '6a', 'd5', '30', '36', 'a5', '38', 'bf', '40', 'a3', '9e', '81', 'f3', 'd7', 'fb'], ['7c', 'e3', '39', '82', '9b', '2f', 'ff', '87', '34', '8e', '43', '44', 'c4', 'de', 'e9', 'cb'], ['54', '7b', '94', '32', 'a6', 'c2', '23', '3d', 'ee', '4c', '95', '0b', '42', 'fa', 'c3', '4e'], ['08', '2e', 'a1', '66', '28', 'd9', '24', 'b2', '76', '5b', 'a2', '49', '6d', '8b', 'd1', '25'], ['72', 'f8', 'f6', '64', '86', '68', '98', '16', 'd4', 'a4', '5c', 'cc', '5d', '65', 'b6', '92'], ['6c', '70', '48', '50', 'fd', 'ed', 'b9', 'da', '5e', '15', '46', '57', 'a7', '8d', '9d', '84'], ['90', 'd8', 'ab', '00', '8c', 'bc', 'd3', '0a', 'f7', 'e4', '58', '05', 'b8', 'b3', '45', '06'], ['d0', '2c', '1e', '8f', 'ca', '3f', '0f', '02', 'c1', 'af', 'bd', '03', '01', '13', '8a', '6b'], ['3a', '91', '11', '41', '4f', '67', 'dc', 'ea', '97', 'f2', 'cf', 'ce', 'f0', 'b4', 'e6', '73'], ['96', 'ac', '74', '22', 'e7', 'ad', '35', '85', 'e2', 'f9', '37', 'e8', '1c', '75', 'df', '6e'], ['47', 'f1', '1a', 

In [11]:
def decrypt(c, k):
    m = to_matrix(c)
    
    #Stage 1-2
    for i in reversed(range(1, len(k))):
        m = ark(m, k[i])
        m = gf_mul(MC_MATRIX_INV, m) if i != len(k) - 1 else m
        m = sb(m, SBOX_INV)
        m = sr_inv(m)
        
    # Stage 3
    m = ark(m, k[0])
    
    return ''.join(to_matrix(''.join(m)))

c = 'e448e574a374d90cc33c22af9b8eab7f'
decrypt(c, k)

'414553206573206d757920666163696c'

In [0]:
###QUIz
# point 9

# Let be m=2b7e151628aed2a6abf7158809cf4f3c and k=9a2bda0ee132ce621fbfacd1896f4721. 
m="2b7e151628aed2a6abf7158809cf4f3c"
ks = to_matrix('9a2bda0ee132ce621fbfacd1896f4721')
#k= "9a2bda0ee132ce621fbfacd1896f4721"
#print(ms)
def to_matrix_inv(mat):
  w1,w2,w3,w4="","","",""
  for e in mat:
    w1+=e[:2]
    w2+=e[2:4]
    w3+=e[4:6]
    w4+=e[6:8]
  return w1+w2+w3+w4


In [14]:
c = ark(m, ks)
print(c)
print(to_matrix_inv(c))

['9ae11f8b', '2b32bf64', 'daceac40', '0e62d12f']
9a2bda0ee132ce621fbfacd18b64402f


In [15]:
''.join(c)

'9ae11f8b2b32bf64daceac400e62d12f'

In [17]:
### 10 Point

#Let be x=19617b67467c866ec20e2e1fc9e08b8e. Find SubBytes(x)

#Do not forget to convert x into a state array and the answer into a hex string.
x="19617b67467c866ec20e2e1fc9e08b8e"
xm = to_matrix(x)
print(xm)

r10 = sb(xm,SBOX)
print(to_matrix_inv(r10))
print(''.join(r10))

['1946c2c9', '617c0ee0', '7b862e8b', '676e1f8e']
d4ef21855a10449f25ab31c0dde13d19
d45a25ddef10abe12144313d859fc019


In [18]:
#### 11 point

## Let be x=19617b67467c866ec20e2e1fc9e08b8e. Find ShiftRows(x)

x="19617b67467c866ec20e2e1fc9e08b8e"
x_m = to_matrix(x)
rta_11 = sr(x_m)
print(to_matrix_inv(rta_11))
print(''.join(rta_11))

197c2e8e460e8b67c2e07b6ec961861f
1946c2c97c0ee0612e8b7b868e676e1f


In [19]:
### 12 point

# Let be x=19617b67467c866ec20e2e1fc9e08b8e. Find MixColumns(x)

x="19617b67467c866ec20e2e1fc9e08b8e"
mc = gf_mul(MC_MATRIX,to_matrix(x))
print(to_matrix_inv(mc))
print(''.join(mc))

8d3127ffe0419fecbcb3b143b71aad2c
8de0bcb73141b31a279fb1adffec432c
